### program settings

In [3]:
import psycopg2
import cv2
from ultralytics import YOLO
from sort.sort import Sort  # Ensure you have SORT installed or use your custom tracking implementation
import numpy as np
from db import Database
import os
from datetime import datetime, timedelta
import uuid
import json
import os

# Load the YOLO model
model = YOLO('/home/annone/ai/survey/data/weights/best.pt')  # Replace with the path to your .pt file

# Input video file (replace with your video path or set to 0 for webcam)
input_video_path = '/home/annone/ai/survey/data/VID_20241115_155636.mp4'  # Replace with the input video file path
output_video_path = '/home/annone/ai/survey/data/video3.mp4'  # Replace with the output video file path
json_data_file = "/home/annone/ai/survey/temp_database_test.json"
class_list = []

{'dbname': 'dev_db', 'user': 'postgres', 'password': 'argus123', 'host': '34.47.226.185', 'port': '8080', 'sslmode': 'disable'}


In [1]:
import json

a = [1,2]
print(json.dumps(a))

[1, 2]


### Save to DB function

In [4]:
conn = Database.get_connection()
cursor = conn.cursor()

In [5]:
def save_to_postgres(thumbnail, class_name, survey_id, geo_coords, distance ):
    query = """
    INSERT INTO "SurveyReport" ("thumbnail", "className", "surveyId", "location", "distance")
    VALUES (%s, %s, %s, %s, %s);
    """
    try:
        data = (f"{thumbnail}", f"{class_name}", f"{survey_id}", f"{geo_coords}", f"{distance}")
        cursor.execute(query, data)
        conn.commit()
        print("Data saved successfully.")

    except Exception as e:
        print(f"Error saving data: {e}")

def log_into_json(file_path=json_data_file, new_data=""):
    try:
        if os.path.exists(file_path):
            with open(file_path, 'r') as json_file:
                try:
                    existing_data = json.load(json_file)
                except json.JSONDecodeError:
                    existing_data = []
        else:
            existing_data = []
        if isinstance(existing_data, list):
            existing_data.append(new_data)
        else:
            raise TypeError("Existing JSON data must be a list to append new data.")
        with open(file_path, 'w') as json_file:
            json.dump(existing_data, json_file, indent=4)
        print(f"Data successfully appended to {file_path}")
    except Exception as e:
        print(f"Error appending to JSON file: {e}")

In [ ]:
import json

# Extract the file's modified time as the initial time
file_modified_time = os.path.getmtime(input_video_path)
initial_time = datetime.fromtimestamp(file_modified_time)
# Read the coordinates from the file
coordinates = []
with open('/home/annone/ai/survey/coordinates_with_timestamp.txt', 'r') as f:  # Replace with the path to your file
    for line in f:
        lat, lon,time_str = [i.split(": ")[1] for i in line.strip().split(',')]
        coordinates.append({
            "timestamp": datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S.%f"),
            "latitude": float(lat),
            "longitude": float(lon)
        })

# Open the input video
cap = cv2.VideoCapture(input_video_path)

# Get the video's width, height, FPS, and frame count
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for .avi output
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize the SORT tracker
tracker = Sort()
known_track_id = []
# Process the video frame by frame
frame_index = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Calculate the current frame's timestamp
    frame_time = initial_time + timedelta(seconds=frame_index / fps)

    # Run the model on the current frame
    results = model(frame)
    class_list = model.names
    # Prepare detections for SORT (format: [x1, y1, x2, y2, confidence])
    detections = []
    for detection in results[0].boxes:
        confidence = detection.conf.item()  # Confidence score
        class_id = int(detection.cls.item())  # Class ID
        class_name = model.names[class_id]  # Class name

        # Skip detections below 30% confidence or class "drainage"
        if confidence < 0.3 or class_name == "drainage":
            continue

        # Get bounding box coordinates
        x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Convert to integers
        detections.append([x1, y1, x2, y2, confidence,class_id])

    # Convert detections to NumPy array
    detections = np.array(detections)

    # Ensure detections array has the correct shape (N, 5)
    if len(detections) == 0:
        detections = np.empty((0, 6))  # Empty array with the correct shape

    # Update the SORT tracker with the current frame's detections
    tracked_objects = tracker.update(detections)

    # Create a copy of the frame for annotation
    annotated_frame = frame.copy()

    # Loop through the tracked objects
    for obj in tracked_objects:
        print(obj)
        # Format: [x1, y1, x2, y2, track_id]
        x1, y1, x2, y2, track_id, class_id = map(int, obj[:6])

        # Draw the bounding box and track ID
        label = f"ID {track_id}"
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(annotated_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Find the nearest timestamp in the coordinates file
        nearest_coord = min(coordinates, key=lambda coord: abs(coord["timestamp"] - frame_time))
        lat, lon = nearest_coord["latitude"], nearest_coord["longitude"]

        # Print the coordinates and lat/lon
        if track_id not in known_track_id:
            # time.sleep(2)
            with open("/home/annone/ai/survey/output.txt", "a") as outfile:
                outfile.write(f"Class: {class_id}, Detection Time: {frame_time}, Lat: {lat}, Lon: {lon}, trackid : {track_id}\n")
                # save_to_postgres(
                #     thumbnail=f"{track_id}.jpg",
                #     class_name=f"{class_list[class_id]}",
                #     geo_coords=json.dumps([lat,lon]),  # Example list of coordinates
                #     survey_id="1",
                #     distance="100"
                # )
                json_data = {
                    "detection_id":f"{uuid.uuid4()}",
                    "thumbnail": f"{track_id}.jpg",
                    "class_name": f"{class_list[class_id]}",
                    "location": [lat,lon],
                    "survey_id": "1",
                    "distance": "100",
                    "track_id": f"{track_id}"
                }
                log_into_json(json_data_file, json_data)
            cv2.imwrite(f"/home/annone/ai/survey/static/images/{track_id}.jpg",annotated_frame)
            print(f"Detection Time: {frame_time}, Lat: {lat}, Lon: {lon}")
            known_track_id.append(track_id)

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Increment the frame index
    frame_index += 1

    # Display the frame (optional)
    # cv2.imshow('Object Tracking', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Database.return_connection(connection=conn)
# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


/home/annone/ai/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


0: 384x640 (no detections), 60.4ms
Speed: 4.1ms preprocess, 60.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.4ms
Speed: 3.0ms preprocess, 28.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.0ms
Speed: 4.2ms preprocess, 27.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 31.3ms
Speed: 2.0ms preprocess, 31.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.2ms
Speed: 2.9ms preprocess, 24.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.8ms
Speed: 2.7ms preprocess, 29.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.7ms
Speed: 1.7ms preprocess, 29.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 crack, 27.8ms
Speed: 1.5ms preprocess, 27.8ms inference,

KeyboardInterrupt: 

In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import uvicorn
import os
import json
import uuid
import cv2
import numpy as np
from ultralytics import YOLO
from sort.sort import Sort
from datetime import datetime, timedelta

app = FastAPI()

# Load the YOLO model
MODEL_PATH = '/home/annone/ai/survey/data/weights/best.pt'  # Update path
model = YOLO(MODEL_PATH)

# JSON data file
JSON_DATA_FILE = "/home/annone/ai/survey/temp_database.json"
COORDINATES_FILE = '/home/annone/ai/survey/coordinates_with_timestamp.txt'
OUTPUT_IMAGE_DIR = "/home/annone/ai/survey/static/images/"
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# Helper: Read and write to JSON file
def log_into_json(file_path, data):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    existing_data.append(data)

    with open(file_path, "w") as f:
        json.dump(existing_data, f, indent=4)

# Helper: Load coordinates with timestamps
def load_coordinates():
    coordinates = []
    with open(COORDINATES_FILE, 'r') as f:
        for line in f:
            lat, lon, time_str = [i.split(": ")[1] for i in line.strip().split(',')]
            coordinates.append({
                "timestamp": datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S.%f"),
                "latitude": float(lat),
                "longitude": float(lon)
            })
    return coordinates

# Route to process video
@app.post("/process-video/")
async def process_video(video: UploadFile = File(...)):
    # Save uploaded video to a temporary location
    input_video_path = f"/tmp/{video.filename}"
    output_video_path = input_video_path.replace(".mp4", "_processed.mp4")

    try:
        with open(input_video_path, "wb") as buffer:
            buffer.write(await video.read())

        # Extract file modified time
        file_modified_time = os.path.getmtime(input_video_path)
        initial_time = datetime.fromtimestamp(file_modified_time)

        # Load coordinates
        coordinates = load_coordinates()

        # Open video file
        cap = cv2.VideoCapture(input_video_path)

        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        # Initialize SORT tracker
        tracker = Sort()
        known_track_id = []
        frame_index = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Calculate frame timestamp
            frame_time = initial_time + timedelta(seconds=frame_index / fps)

            # Run YOLO model
            results = model(frame)
            class_list = model.names

            # Prepare detections for SORT
            detections = []
            for detection in results[0].boxes:
                confidence = detection.conf.item()
                class_id = int(detection.cls.item())
                class_name = model.names[class_id]

                if confidence < 0.3 or class_name == "drainage":
                    continue

                x1, y1, x2, y2 = map(int, detection.xyxy[0])
                detections.append([x1, y1, x2, y2, confidence, class_id])

            detections = np.array(detections) if len(detections) > 0 else np.empty((0, 6))

            # Update tracker
            tracked_objects = tracker.update(detections)

            # Annotate frame
            annotated_frame = frame.copy()

            for obj in tracked_objects:
                x1, y1, x2, y2, track_id, class_id = map(int, obj[:6])
                label = f"ID {track_id}"
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                nearest_coord = min(coordinates, key=lambda coord: abs(coord["timestamp"] - frame_time))
                lat, lon = nearest_coord["latitude"], nearest_coord["longitude"]

                if track_id not in known_track_id:
                    json_data = {
                        "detection_id": str(uuid.uuid4()),
                        "thumbnail": f"{track_id}.jpg",
                        "class_name": f"{class_list[class_id]}",
                        "location": [lat, lon],
                        "survey_id": "1",
                        "distance": "100",
                        "track_id": str(track_id)
                    }
                    log_into_json(JSON_DATA_FILE, json_data)
                    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, f"{track_id}.jpg"), annotated_frame)
                    known_track_id.append(track_id)

            out.write(annotated_frame)
            frame_index += 1

        # Release resources
        cap.release()
        out.release()

        return JSONResponse(content={"message": "Video processed successfully", "output_video": output_video_path})

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred: {str(e)}")

    finally:
        if os.path.exists(input_video_path):
            os.remove(input_video_path)

# Run the app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
